# SQL to circuit ansätze

This notebook implements the part of the algorithm which translates join order benchmark (JOB) and the simplified join order benchmark queries into pregroup diagrams and pregroup diagrams into circuit ansätze. The simple example can be found in `sql_to_circuit_simple_example` notebook.

The following code generates diagrams for all the SELECT-FROM-WHERE queries in the join order benchmark and their simplified versions. Running the code will take some time and it also works as a test package for the code. The diagrams are already generated in the folders `join-order-benchmark-diagrams` and `simplified-JOB-diagrams`.

Unfortunalyte, JOB queries produce too large circuits for quantum computing resources that we have available. That is why we created the set of simplified queries. As in the data generation case, this notebook is for reproducibility reasons and the user does not need to rerun this if they do not want to change the underlying queries or the mappings.

In [6]:
from antlr4 import *
from SQLiteLexer import SQLiteLexer
from SQLiteParser import SQLiteParser
from SQLiteParserListener import SQLiteParserListener
import json
import os
import glob
from pathlib import Path
from discopy import Ty, Box, Functor
from functools import reduce
from discopy.utils import dumps, loads
from multiprocessing import Pool

import diagramGenerators

this_folder = os.path.abspath(os.getcwd())

In [7]:
query_path_job = "\\join-order-benchmark-queries\\[0-9]*.sql"
query_path_training = "\\training_queries\\[0-9]*.sql"
query_path_test = "\\test_queries\\[0-9]*.sql"

cfg_folder_name_job = "join-order-benchmark-diagrams//cfg-diagrams"
cfg_folder_name_training = "simplified-JOB-diagrams//cfg-diagrams"
cfg_folder_name_test = "simplified-JOB-diagrams//cfg-diagrams"

pregroup_folder_job = "join-order-benchmark-diagrams//pregroup-diagrams"
pregroup_folder_training = "simplified-JOB-diagrams//pregroup-diagrams"
pregroup_folder_test = "simplified-JOB-diagrams//pregroup-diagrams"

pregroup_cup_removed_folder_job = "join-order-benchmark-diagrams//cup-removed-pregroup-diagrams"
pregroup_cup_removed_folder_training = "simplified-JOB-diagrams//cup-removed-pregroup-diagrams"
pregroup_cup_removed_folder_test = "simplified-JOB-diagrams//cup-removed-pregroup-diagrams"

circuit_job = "join-order-benchmark-diagrams//circuits"
circuit_training = "simplified-JOB-diagrams//circuits//training"
circuit_test = "simplified-JOB-diagrams//circuits//test"

In [13]:
def split(list_a, chunk_size):
    if list_a == []:
        return []
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]

## Transformation 1: SQL to context-free grammar diagrams

The following cells execute the transformations in parallel.

In [9]:
queries_job = glob.glob(this_folder + query_path_job)

if __name__ ==  '__main__':
    num_processors = 8
    chunks = split(queries_job, int(len(queries_job)/num_processors))
    p = Pool(processes = num_processors)
    p.starmap(diagramGenerators.create_CFG_diagrams, [(chunk, cfg_folder_name_job) for chunk in chunks])
    p.close()
    p.join()

In [10]:
queries_training = glob.glob(this_folder + query_path_training)

if __name__ ==  '__main__':
    num_processors = 8
    chunks = split(queries_training, int(len(queries_training)/num_processors))
    p = Pool(processes = num_processors)
    p.starmap(diagramGenerators.create_CFG_diagrams, [(chunk, cfg_folder_name_training) for chunk in chunks])
    p.close()
    p.join()

In [14]:
queries_test = glob.glob(this_folder + query_path_test)

if __name__ ==  '__main__':
    num_processors = 8
    chunks = split(queries_test, int(len(queries_test)/num_processors))
    p = Pool(processes = num_processors)
    p.starmap(diagramGenerators.create_CFG_diagrams, [(chunk, cfg_folder_name_test) for chunk in chunks])
    p.close()
    p.join()

## Transformation 2: Context-free grammar diagrams to pregroup grammar diagrams

The following cells execute the transformations in parallel.

In [15]:
cfg_diagrams = glob.glob(this_folder + "\\" + cfg_folder_name_job + "\\[0-9]*.json")

if __name__ ==  '__main__':
    num_processors = 8
    chunks = split(cfg_diagrams, int(len(cfg_diagrams)/num_processors))
    p = Pool(processes = num_processors)
    p.starmap(diagramGenerators.create_pregroup_grammar_diagrams, [(chunk, pregroup_folder_job) for chunk in chunks])
    p.close()
    p.join()

In [16]:
cfg_diagrams_training = glob.glob(this_folder + "\\" + cfg_folder_name_training + "\\[0-9]*.json")

if __name__ ==  '__main__':
    num_processors = 8
    chunks = split(cfg_diagrams_training, int(len(cfg_diagrams_training)/num_processors))
    p = Pool(processes = num_processors)
    p.starmap(diagramGenerators.create_pregroup_grammar_diagrams, [(chunk, pregroup_folder_training) for chunk in chunks])
    p.close()
    p.join()

In [17]:
cfg_diagrams_test = glob.glob(this_folder + "\\" + cfg_folder_name_test + "\\[0-9]*.json")

if __name__ ==  '__main__':
    num_processors = 8
    chunks = split(cfg_diagrams_test, int(len(cfg_diagrams_test)/num_processors))
    p = Pool(processes = num_processors)
    p.starmap(diagramGenerators.create_pregroup_grammar_diagrams, [(chunk, pregroup_folder_test) for chunk in chunks])
    p.close()
    p.join()

## Transformation 3: pregroup diagram rewriting, cup removal and simplification

The following cells execute the transformations in parallel.

In [ ]:
def cup_remove_arrow_mapping(box):
    if box.name.lower() == 'select':
        return box
    elif not box.cod:
        domain = box.dom
        raised_leg = Ty(domain[0])
        new_domain = reduce(lambda x, y : x @ Ty(y), domain[1:], Ty())
        new_box = Id(raised_leg) @ Box(box.name, new_domain, raised_leg.l)\
        >> Cup(raised_leg, raised_leg.l)
        return new_box
    return box

def cup_remove_arrow_mapping2(box):
    if box.cod == box.dom == Ty('n'):
        return Id(box.cod)
    return box

cup_removal_functor = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping(f))
cup_removal_functor2 = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping2(f))

In [ ]:
def remove_cups_and_simplify(pregroup_folder_name, cup_removed_pregroup_folder_name):
    pregroup_diagrams = glob.glob(this_folder + "\\" + pregroup_folder_name + "\\[0-9]*.json")
    
    cupless_pregroup_diagram = cup_removal_functor(pregroup_diagram.normal_form()).normal_form()
    cupless_pregroup_diagram = cup_removal_functor2(cupless_pregroup_diagram).normal_form()
    width = cupless_pregroup_diagram.width()
    height = cupless_pregroup_diagram.depth()
    dim = 3*max(width, height)
    cupless_pregroup_diagram.draw(figsize=(dim, dim))

## Transformation 4: pregroup diagrams to circuit ansätze

The following cells execute the transformations in parallel.

In [ ]:
n, s = Ty('n'), Ty('s')
ansatz = IQPAnsatz({n: 1, s: 2}, n_layers=1, n_single_qubit_params=3)
    
def create_circuit_ansatz():
    
    circuit_diagram = ansatz(cupless_pregroup_diagram)
    width = circuit_diagram.width()
    height = circuit_diagram.depth()
    dim = 3*max(width, height)
    circuit_diagram.draw(figsize=(dim, dim)) #, path = this_folder + "\\figures\\circuit_figure.png")